## Bilateral filter 

###### Author: Cristina Vrinceanu (University of Nottingham), Americo Ambrozio (DEIMOS/ESA)

Bilateral filter consists in a non-iterative method for removing noise in images, while preserving edges. The filter acts in both the range and domain of the underlining function, hence, the filter resolves both the photometric and geometric aspects. More information can be found here: http://homepages.inf.ed.ac.uk/rbf/CVonline/LOCAL_COPIES/MANDUCHI1/Bilateral_Filtering.html#Index. Since the OpenCV library already comes with an implementation of the Bilateral filter, there is no need to redefine it. For information on parameter specifications, please check the documentation: https://docs.opencv.org/2.4/modules/imgproc/doc/filtering.html?highlight=bilateralfilter#bilateralfilter

In [ ]:
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
from skimage import exposure
import math
import cv2 as cv
import glob
import os
import subprocess

In [ ]:
##subprocess.run(["exiftool", "-ifd0:modeltiepoint=", output_file])#, stdout=subprocess.DEVNULL)
##subprocess.run(["exiftool", "-TagsFromFile", input_file, "-ifd0:modeltiepoint<modeltiepoint", output_file])#, stdout=subprocess.DEVNULL)

In [ ]:
def bilateral_filter(input_file, output_file):
    print('Begin filtering file ' + input_file)
    
    # inputing the file
    sar_image=gdal.Open(input_file)
    sar_band=sar_image.GetRasterBand(1)
    sar_band.GetMetadata()
    img_array=sar_band.ReadAsArray()
    img_array = img_array.astype(np.float32)
    [cols, rows]=img_array.shape
      
    # filtering
    filtered_image = cv.bilateralFilter(img_array,5,80,80)
    
    # writing output file
    driver = gdal.GetDriverByName("GTiff")
    output = driver.Create(output_file, rows, cols, 1, gdal.GDT_Float32)
    proj = output.SetGeoTransform(sar_image.GetGeoTransform())  
    output.SetProjection(sar_image.GetProjection())
    output.SetGCPs(sar_image.GetGCPs(), "4326")
    geoband = output.GetRasterBand(1)
    geoband.WriteArray(filtered_image)
    output.GetRasterBand(1).SetNoDataValue(-9999)
    output.FlushCache()
    output = None
    band=None
    print('Created file ' + output_file)
    
    print('End filtering file ' + input_file)

In [ ]:
for input_file in glob.glob('/home/cristina/Seeps/Workflow/Filters/ROI/*/NRCS/*tif'):
    print(input_file)
    output_file = input_file.split(os.sep)
    print(output_file)
    output_file[-1] = output_file[-1][:-4] + '_Bil.tif'
    print(output_file)
    output_file = output_file[:-2] + ['Filters'] + ['Bilateral'] + output_file[-2:]
    print(output_file)
    output_file = os.sep.join(output_file)
    print(output_file)
    bilateral_filter(input_file, output_file)